### SpeechAnalyzer

## Progetto Technologies for Advanced Programming 2019/2020

- Aldo Fiorito X81000447
- Francesco Petrosino X81000533\
Studenti presso Università di Catania cdl Informatica

## Idea e scopo di SpeechAnalyzer
L'idea è nata osservando una community di persone all'interno dell'applicativo VOIP per le chat vocali Discord.\ La domanda principale posta è stata "Chi è il più chiacchierone del gruppo?"

Bisogna fare prima una premessa ...

![](train.jpg)

## Da qui SpeechAnalyzer

Fornisce statistiche e metriche su numerosi flussi vocali all'interno della propria rete

## Architettura dell'applicativo

### In breve
1. L'utente si registra all'applicativo
2. Un file json costruito dal suo flusso vocale viene mandato a un kafka-producer, avente come topic un campo   specificato dall'user
3. Spark , essendo anche kafka-consumer , processa ed elabora altre informazioni dal dato e costruisce un datafraframe attraverso sparksql 
4. La nuova struttura creata viene indicizzata su elasticsearch
5. Kibana si occupa di ottenere le metriche e le statistiche dai dati indicizzati attraverso un'interfaccia user-end

## Flusso operativo

L'utente,tramite una form, viene fatto registrare all'applicativo inserendo il proprio nome,la propria compagnia( che sarà il topic kafka) e la propria lingua d'appartenenza.\
**Perchè la lingua?**
Perchè l'analizzatore riesce ad elaborare qualiasi lingua ( al momento le 5 principali ).\
Ogni input verrà tradotto nella stessa lingua

### PreProcessing del dato
Un primo step di preprocessing dell'input vocale è quello di filtraggio del testo.\
Attraverso un file in locale di "badwords" verranno sostituite quelle parole considerate blasfeme o offensive
Come secondo step il testo filtrato viene tradotto in inglese


### Kafka
E' la prima tecnologia della pipeline che andremo ad utilizzare
( spiegare kafka e zookeaper )






Il topic-kafka verrà creato da codice python:

In [ ]:
admin_client = KafkaAdminClient(
                   bootstrap_servers="192.168.1.28:9092", 
                   client_id='test'
               )
topic_list = [NewTopic(name=topicKafka, num_partitions=1, replication_factor=1)]
admin_client.create_topics(new_topics=topic_list, validate_only=False)

Il file json inviato al kafka-producer avrà questa struttura :\\ 

{"name":"saro" , "message" : "Microsoft has released security updates ","topic" : "Tech","language" : "en","id":"75df85bf"}

Dove topic si riferisce al predict ottenuto dall'algoritmo di kmeans implementato.

### Modello Machine Learning e K-means

![](ml.png)

L'immagine appena mostrata rappresenta il modello di kmeans utilizzato e salvato in locale attraverso la libreria "pickle".
Per inizializzare il modello è stato dato in pasto un dataset di news ottenuto al seguente indirizzo URL
http://mlg.ucd.ie/datasets/bbc.html.
Le news appartengono a 5 categorie diverse: business, entertainment, politics, sport, tech.
Questi documenti raw devono essere processati prima di essere passati all'algoritmo.
StopWords,rimuovere le parole insignificanti
Stemming,portare le parole alla root form
tokenization , per

### Salvataggio e restore del modello ML
k_model=MLAPI()
Pkl_Filename = "kmodel.pkl"
    with open(os.path.join('./python/bin/MachineL',Pkl_Filename), 'wb') as file:  
        pickle.dump(k_model,file)
        
with open('./python/bin/MachineL/kmodel.pkl', 'rb') as f:
        model = pickle.load(f)      